# Clustering Code Along

We'll be working with a real data set about seeds, from UCI repository: https://archive.ics.uci.edu/ml/datasets/seeds.

The examined group comprised kernels belonging to three different varieties of wheat: Kama, Rosa and Canadian, 70 elements each, randomly selected for 
the experiment. High quality visualization of the internal kernel structure was detected using a soft X-ray technique. It is non-destructive and considerably cheaper than other more sophisticated imaging techniques like scanning microscopy or laser technology. The images were recorded on 13x18 cm X-ray KODAK plates. Studies were conducted using combine harvested wheat grain originating from experimental fields, explored at the Institute of Agrophysics of the Polish Academy of Sciences in Lublin. 

The data set can be used for the tasks of classification and cluster analysis.


Attribute Information:

To construct the data, seven geometric parameters of wheat kernels were measured: 
1. area A, 
2. perimeter P, 
3. compactness C = 4*pi*A/P^2, 
4. length of kernel, 
5. width of kernel, 
6. asymmetry coefficient 
7. length of kernel groove. 
All of these parameters were real-valued continuous.

Let's see if we can cluster them in to 3 groups with K-means!

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('clustercodealong').getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans

# Loads data.
dataset = spark.read.csv("seeds_dataset.csv",header=True,inferSchema=True)

In [4]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [8]:
dataset.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [5]:
dataset.describe().toPandas()

,summary,area,perimeter,compactness,length_of_kernel,width_of_kernel,asymmetry_coefficient,length_of_groove
0,count,210,210,210,210,210,210,210
1,mean,14.847523809523816,14.559285714285718,0.8709985714285714,5.628533333333335,3.258604761904762,3.7001999999999997,5.408071428571429
2,stddev,2.9096994306873647,1.3059587265640225,0.023629416583846364,0.44306347772644983,0.3777144449065867,1.5035589702547392,0.49148049910240543
3,min,10.59,12.41,0.8081,4.899,2.63,0.765,4.519
4,max,21.18,17.25,0.9183,6.675,4.033,8.456,6.55


## Format the Data

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')

In [16]:
prepared_data = vec_assembler.transform(dataset)

## Scale the Data
It is a good idea to scale our data to deal with the curse of dimensionality: https://en.wikipedia.org/wiki/Curse_of_dimensionality

In [17]:
from pyspark.ml.feature import StandardScaler

In [18]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [19]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(prepared_data)

In [20]:
# Normalize each feature to have unit standard deviation.
final_data = scalerModel.transform(final_data)

In [33]:
prepared_data.select('features').limit(5).collect()

[Row(features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22])),
 Row(features=DenseVector([14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956])),
 Row(features=DenseVector([14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825])),
 Row(features=DenseVector([13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805])),
 Row(features=DenseVector([16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175]))]

In [34]:
final_data.select('scaledFeatures').limit(5).collect()

[Row(scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621])),
 Row(scaledFeatures=DenseVector([5.1139, 11.1566, 37.2883, 12.5354, 8.8241, 0.6771, 10.0838])),
 Row(scaledFeatures=DenseVector([4.9112, 10.789, 38.2997, 11.9419, 8.8347, 1.7951, 9.8173])),
 Row(scaledFeatures=DenseVector([4.7565, 10.6742, 37.8977, 12.0163, 8.9459, 1.5024, 9.7766])),
 Row(scaledFeatures=DenseVector([5.547, 11.4782, 38.232, 12.7702, 9.4304, 0.9012, 10.5294]))]

## Train the Model and Evaluate

In [48]:
# Trains a k-means model.
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [49]:
# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(final_data)
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 429.07559671506715


 - K=2 WSSSE=656
 - K=3 WSSSE=429
 - K=4 WSSSE=382
 - K=5 WSSSE=360

In [53]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 4.87257659 10.88120146 37.27692543 12.3410157   8.55443412  1.81649011
 10.32998598]
[ 4.06105916 10.13979506 35.80536984 11.82133095  7.50395937  3.27184732
 10.42126018]
[ 6.31670546 12.37109759 37.39491396 13.91155062  9.748067    2.39849968
 12.2661748 ]


In [80]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         0|
|         0|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         2|
+----------+
only showing top 20 rows



Now you are ready for your consulting Project!
# Great Job!